In [48]:
from portiloop_software.portiloop_python.ANN.data.mass_data import SingleSubjectDataset, read_pretraining_dataset, read_spindle_trains_labels
from portiloop_software.portiloop_python.ANN.utils import get_configs

In [3]:
import numpy as np

file1 = '/project/portiloop-training/portiloop_software/dataset/6_segListSrcDataLoc_p1.txt'
file2 = '/project/portiloop-training/portiloop_software/dataset/7_segListSrcDataLoc_p2.txt'

In [14]:
data1 = np.genfromtxt(file1, delimiter='\t', dtype=str)[1:, [1, 4]]
data2 = np.genfromtxt(file2, delimiter='\t', dtype=str)[1:, [1, 4]]

In [19]:
all_segments = np.concatenate((data1, data2), axis=0)
all_segments.shape

(749, 2)

This line gets the starting index in seconds for each segment for each subject and puts it in a dictionary where the key is the subject id and the values are lists of the starting indexes. Each segment will hen be exactly 115 seconds.

In [43]:
segments_dict = {subject: (all_segments[np.where(all_segments[:, 0] == subject)[0], 1].astype(float) * 250).astype(int).tolist() for subject in np.unique(all_segments[:, 0])}

Load the Wamsley data

In [47]:
experiment_name = 'MASS'
seed = 42

config = get_configs(experiment_name, False, seed)

data = read_pretraining_dataset(config['MASS_dir'])
labels = read_spindle_trains_labels(config['old_dataset'])

In [50]:
from portiloop_software.portiloop_python.ANN.data.mass_data import read_sleep_staging_labels

ss_labels = read_sleep_staging_labels(config['path_dataset'])


In [54]:
wamsley_labels = {}
all_keys = list(data.keys())
for subject in all_keys:
    try:
        dataset = SingleSubjectDataset(subject, data=data, labels=labels, config=config, ss_labels=ss_labels)  
        wamsley_labels[subject] = dataset.full_labels
    except KeyError:
        continue

In [57]:
for wamsley_label in wamsley_labels.keys():
    if wamsley_label not in segments_dict.keys():
        continue

    

01-01-0034
01-01-0039
01-02-0007
01-02-0008
01-02-0009
01-02-0016
01-02-0018
01-03-0055
01-05-0021
01-05-0026


Load the MODA Data

In [58]:
subjects_path = '/project/portiloop-training/portiloop_software/dataset/subject_sequence_full_big.txt'
data_path = '/project/portiloop-training/portiloop_software/dataset/dataset_classification_full_big_250_matlab_standardized_envelope_pf.txt'

In [69]:
import pandas as pd

data = pd.read_csv(data_path, header=None).to_numpy()


In [60]:
subjects = np.genfromtxt(subjects_path, dtype=str)

In [65]:
mask = (data[:, 3] != -1) & (data[:, 3] != -2)
data_filtered = data[mask]

In [68]:
len(data_filtered) / 115 / 250

695.0

In [79]:
split_data = np.array(np.split(data_filtered[:, -1], int(len(data_filtered) / (115 * 250))))
split_data.shape

(695, 28750)

In [83]:
subjects.shape

(162, 3)

In [99]:
moda_index_dict = {}
for subject in subjects[:, 0]:
    s = subjects[subjects[:, 0] == subject][:, 1:].astype(int)[0]
    moda_index_dict[subject] = list(range(s[0], s[1]))

In [103]:
moda_labels = {}

for subject in moda_index_dict.keys():
    moda_labels[subject] = np.hstack(split_data[moda_index_dict[subject]])

In [105]:
moda_labels['01-04-0018'].shape

(86250,)

Putting everything together:

In [107]:
count = 0
final_wamsley_labels = []
final_moda_labels = []
for subject in moda_labels.keys():
    if subject not in wamsley_labels.keys():
        continue

    # Get the necessary indexes of the wamsley labels


63